# Simple DADVI example

This notebook runs DADVI, computes LR covariances, and compares against NUTS for one of the simple models included in PyMC.

In [ ]:
import pymc as pm
import pytensor
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from jax.config import config

# This makes sure we use double precision
config.update("jax_enable_x64", True)

# The next import is the DADVI library.
# This is the most convenient function to run.
from dadvi.pymc.jax_api import fit_pymc_dadvi_with_jax

In [ ]:
# This is a good example model since it's built into PyMC
def fetch_radon_model():

    data = pd.read_csv(pm.get_data("radon.csv"))
    data["log_radon"] = data["log_radon"].astype(pytensor.config.floatX)
    county_names = data.county.unique()
    county_idx = data.county_code.values.astype("int32")

    n_counties = len(data.county.unique())

    with pm.Model() as hierarchical_model:
        # Hyperpriors for group nodes
        mu_a = pm.Normal("mu_a", mu=0.0, sigma=100.0)
        sigma_a = pm.HalfNormal("sigma_a", 5.0)
        mu_b = pm.Normal("mu_b", mu=0.0, sigma=100.0)
        sigma_b = pm.HalfNormal("sigma_b", 5.0)

        # Intercept for each county, distributed around group mean mu_a
        # Above we just set mu and sd to a fixed value while here we
        # plug in a common group distribution for all a and b (which are
        # vectors of length n_counties).
        a = pm.Normal("a", mu=mu_a, sigma=sigma_a, shape=n_counties)
        # Intercept for each county, distributed around group mean mu_a
        b = pm.Normal("b", mu=mu_b, sigma=sigma_b, shape=n_counties)

        # Model error
        eps = pm.HalfCauchy("eps", 5.0)

        radon_est = a[county_idx] + b[county_idx] * data.floor.values

        # Data likelihood
        radon_like = pm.Normal(
            "radon_like", mu=radon_est, sigma=eps, observed=data.log_radon
        )

    return hierarchical_model

In [ ]:
# Define your PyMC model as usual:
# Here we could use any PyMC model instead!
m = fetch_radon_model()

In [ ]:
# Fit with DADVI using 30 fixed draws.
model_result = fit_pymc_dadvi_with_jax(m, num_fixed_draws=30)

In [ ]:
# If we do this, we get draws from the mean field estimate
mean_field_draws = model_result.get_posterior_draws_mean_field()

# We could also look at the mean field means and standard deviations
mf_means = model_result.get_posterior_means()
mf_sds = model_result.get_posterior_standard_deviations_mean_field()

# All three are formatted the same way:
print(mean_field_draws.keys(), '\n')
print(mean_field_draws['b'].shape, '\n')

print(mf_means.keys(), '\n')
print(mf_sds.keys(), '\n')
print(mf_means['b'].shape, '\n')

In [ ]:
# The posterior draws differ from the MF optimum only by sampling error.  The advantage
# of the draws is in computing expectations of nonlinear functions of the parameters.
plt.plot(mf_means['b'], np.mean(mean_field_draws['b'], axis=0), '.')
plt.xlabel('Mean field posterior mean')
plt.xlabel('Mean of posterior draws')

# LRVB correction

- `lrvb_sd` is the LRVB estimate of the posterior SD
- `freq_sd` is the LRVB estimate of the SD due to the fixed draws
- `n_hvp_calls` is the number of hvp calls that were required to compute this

In [ ]:
# Here we use the identity function, but something else works too via the delta method.
# This also computes the frequentist SD estimate.
lrvb_corrected = model_result.get_frequentist_sd_and_lrvb_correction_of_scalar_valued_function(lambda x: x['b'][0])
print(lrvb_corrected.keys())

In [ ]:
print(f'LR sd =\t{lrvb_corrected["lrvb_sd"]}')
print(f'MF sd =\t{mf_sds["b"][0]}')

In [ ]:
lr_draws = np.random.normal(loc=lrvb_corrected['mean'], scale=lrvb_corrected['lrvb_sd'], size=(1000, ))

# Run the same model with PyMC NUTS

In [ ]:
with m as model:
    nuts_res = pm.sample()

In [ ]:
# Flatten NUTS into a vector
example_variable = nuts_res.posterior['b'].values
num_chains, num_draws, _ = example_variable.shape
reshaped = example_variable.reshape(num_chains * num_draws, -1)

## Compare NUTS with mean field and linear response

In [ ]:
# Compare against mean field
plt.hist(reshaped[:, 0], density=True, alpha=0.5, label='NUTS (MCMC)')
plt.hist(mean_field_draws['b'][:, 0], density=True, alpha=0.5, label='Mean field')
plt.hist(lr_draws, density=True, alpha=0.5, label='Linear response')
plt.xlabel('Draws of b[0]')
plt.legend()